## Projeto Final - Curso de Python Coderhouse

In [57]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [58]:
import pandas as pd
import requests
import sqlite3
import numpy as np
from plyer import notification 
from datetime import datetime
from dateutil.relativedelta import relativedelta

### Criação do Alerta

In [59]:
def alerta(nivel, base, etapa, erro=""):
    '''Função que gera um alerta de erro devido à falha no carregamento de base de dados, com informação do tipo de alerta e horário da falha'''

    data_atual = str(datetime.now())

    mensagem = f"Falha no carregamento da base {base} na etapa {etapa} em {data_atual}"

    #Nível de alerta médio não compromete a execução do código, mas haverá falhas no tratamento
    if nivel == 2:
        title = 'ATENÇÃO: Alerta Médio'
    #Nível de alerta alto compromete a execução do código, pois se refere à falha no carregamento dos dados
    elif nivel  == 3:
        title = 'ATENÇÃO: Alerta Alto'
    else:
        print("Nivel",nivel,"não disponível!")
        
    notification.notify(
            title=title,
            message=mensagem,
            app_name='alerta',
            timeout=10
        )
            

### Conexão às APIs

In [60]:
def get_json_api(url):
    '''Função requisição GET, retorna o json de saida da API da url e informa se houve erro'''

    response = requests.get(url)

    if response.status_code == 200:
        data_json = response.json()
        return data_json 
    

In [61]:
#Tabela 1 - Taxa Selic
def get_selic(data_inicial, data_final):
    '''Requisição da tabela de valores definidos da Selic Anual por mês, filtrada pelo período indicado pelas datas no formato "DD/MM/AAAA"'''
    
    url_selic = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.4189/dados?formato=json&dataInicial={data_inicial}&dataFinal={data_final}"
  
    data_json = get_json_api(url_selic)
    tabela_selic = pd.DataFrame(data_json)

    return tabela_selic


In [62]:
#Tabela 2 - IPCA
def get_ipca(data_inicial, data_final):
    '''Requisição da tabela de valores do IPCA por mês, filtrada pelo período indicado pelas datas no formato "DD/MM/AAAA"'''
    
    url_ipca = f"http://api.bcb.gov.br/dados/serie/bcdata.sgs.433/dados?formato=json&dataInicial={data_inicial}&dataFinal={data_final}"

    data_json = get_json_api(url_ipca)
    tabela_ipca = pd.DataFrame(data_json)

    return tabela_ipca


In [63]:
#Tabela 3 - IGP-M
def get_igpm(data_inicial, data_final):
    '''Requisição da tabela de valores do IGP-M por mês, filtrada pelo período indicado pelas datas no formato "DD/MM/AAAA"'''
    
    url_igpm = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.189/dados?formato=json&dataInicial={data_inicial}&dataFinal={data_final}"

    data_json = get_json_api(url_igpm)
    tabela_igpm = pd.DataFrame(data_json)

    return tabela_igpm


### Banco de Dados

In [64]:
def tabelas_bd():
    '''Retorna um dataframe com as tabelas do banco de dados'''

    conn = sqlite3.connect('coderhouse.db')

    #Executar uma consulta que retorna as informações do esquema do banco de dados
    query = "SELECT name FROM sqlite_master WHERE type='table'"
    schema = pd.read_sql_query(query, conn)

    conn.close()

    return schema

def salva_bd(df, nome_tabela):
    '''Salva dataframe df na tabela nome_tabela'''
    conn = sqlite3.connect('coderhouse.db')

    #Escrever o DataFrame na tabela 'nome_tabela'
    df.to_sql(nome_tabela, conn, if_exists='replace', index=False)

    conn.close()

    return True

def carrega_bd(nome_tabela):
    '''Carrega tabela nome_tabela num dataframe'''
    conn = sqlite3.connect('coderhouse.db')

    #Executar uma consulta SELECT na tabela e converter em um DataFrame
    query = f"SELECT * FROM {nome_tabela}"
    df = pd.read_sql(query, conn)

    conn.close()

    return df

### Extração das APIs

In [71]:
def etapa_extracao(data_inicial, data_final):
    '''Extrai os dados das APIs selecionadas e salva no banco de dados'''
    
    #1 - Selic
    #roda função de requisição da api selic e salva no banco de dados
    try:
        df_selic = get_selic(data_inicial, data_final)
        salva_bd(df_selic,"selic_url")
    except Exception as e:
        alerta(nivel = 3, 
               base = "selic_url", 
               etapa = "EXTRACAO", 
               erro=e)
    
    #2 - IPCA
    #roda função de requisição da api ipca e salva no banco de dados
    try:
        df_ipca = get_ipca(data_inicial, data_final)
        salva_bd(df_ipca,"ipca_url")
    except Exception as e:
        alerta(nivel = 3, 
               base = "ipca_url", 
               etapa = "EXTRACAO", 
               erro=e)    
    #3 - IGP-M
    #roda função de requisição da api igpm e salva no banco de dados
    try:
        df_igpm = get_igpm(data_inicial, data_final)
        salva_bd(df_igpm,"igpm_url")
    except Exception as e:
        alerta(nivel = 3, 
               base = "igpm_url", 
               etapa = "EXTRACAO", 
               erro=e)            


### Transformação e Tratamento dos Dados

In [66]:
def etapa_transformacao():
    
    try:
        ##Selic##
        #Carrega e ajusta o nome da coluna de valor da selic do mes
        df_selic = carrega_bd("selic_url")
        df_selic = df_selic.rename(columns={'valor':'valor_selic'})
        salva_bd(df_selic,"tabela_selic")

        ##IPCA##
        #Carrega e ajusta o nome da coluna de valor do ipca do mes
        df_ipca = carrega_bd("ipca_url")
        df_ipca = df_ipca.rename(columns={'valor':'valor_ipca'})
        salva_bd(df_ipca,"tabela_ipca")

        #Converte o valor do ipca para float e adiciona uma coluna com o ipca acumulado de 12 meses
        df_ipca['valor_ipca'] = df_ipca['valor_ipca'].astype(float)
        df_ipca['aux'] = 1 + (df_ipca['valor_ipca'] / 100)
        df_ipca['ipca_acum_12'] = df_ipca['aux'].rolling(12).apply(np.prod, raw=True) - 1

        #Ajuste para manter o ipca acumulado no mesmo formato do ipca mensal, com 2 casas decimais
        df_ipca['ipca_acum_12'] = df_ipca['ipca_acum_12'] * 100
        df_ipca['ipca_acum_12'] = df_ipca['ipca_acum_12'].round(2)
        
        #Drop da coluna aux utilizada no cálculo do ipca acumulado
        df_ipca = df_ipca.drop(columns='aux')

        ##IGP-M##
        #Carrega e ajusta o nome da coluna de valor do igpm do mes
        df_igpm = carrega_bd("igpm_url")
        df_igpm = df_igpm.rename(columns={'valor':'valor_igpm'})
        salva_bd(df_igpm,"tabela_igpm")
        
        #Converte o valor do igpm para float e adiciona uma coluna com o igpm acumulado de 12 meses
        df_igpm['valor_igpm'] = df_igpm['valor_igpm'].astype(float)
        df_igpm['aux'] = 1 + (df_igpm['valor_igpm'] / 100)
        df_igpm['igpm_acum_12'] = df_igpm['aux'].rolling(12).apply(np.prod, raw=True) - 1

        #Ajuste para manter o igpm acumulado no mesmo formato do igpm mensal, com 2 casas decimais
        df_igpm['igpm_acum_12'] = df_igpm['igpm_acum_12'] * 100
        df_igpm['igpm_acum_12'] = df_igpm['igpm_acum_12'].round(2)
        
        #Drop da coluna aux utilizada no cálculo do igpm acumulado
        df_igpm = df_igpm.drop(columns='aux')    

        #Fazendo os joins das tabelas
        df_selic = df_selic.merge(df_ipca, on=['data'],how='left')
        df_selic = df_selic.merge(df_igpm, on=['data'],how='left')

        salva_bd(df_selic,"tabela_indices")
        print("tabela_indices salva")
    except Exception as e:
        alerta(nivel = 2, 
               base = "tabela_indices", 
               etapa = "TRANSFORMACAO", 
               erro=e)


### Execução

In [67]:
#Tratamento para consultar 3 anos de dados das tabelas, considerando data atual
data_inicial = datetime.now() - relativedelta(years=3)
data_inicial_ajust = data_inicial.strftime("%d/%m/%Y")

data_final = datetime.now()
data_final_ajust = data_final.strftime("%d/%m/%Y")

print(data_inicial_ajust)
print(data_final_ajust)
etapa_extracao(data_inicial_ajust,data_final_ajust)

06/09/2021
06/09/2024


In [68]:
etapa_transformacao()

tabela_indices salva


### Carregamento da Base no Banco

In [69]:
df = carrega_bd("tabela_indices")
print(df)

          data valor_selic  valor_ipca  ipca_acum_12  valor_igpm  igpm_acum_12
0   01/09/2021        5.43        1.16           NaN       -0.64           NaN
1   01/10/2021        6.30        1.25           NaN        0.64           NaN
2   01/11/2021        7.65        0.95           NaN        0.02           NaN
3   01/12/2021        8.76        0.73           NaN        0.87           NaN
4   01/01/2022        9.15        0.54           NaN        1.82           NaN
5   01/02/2022       10.49        1.01           NaN        1.83           NaN
6   01/03/2022       11.15        1.62           NaN        1.74           NaN
7   01/04/2022       11.65        1.06           NaN        1.41           NaN
8   01/05/2022       12.51        0.47           NaN        0.52           NaN
9   01/06/2022       12.89        0.67           NaN        0.59           NaN
10  01/07/2022       13.15       -0.68           NaN        0.21           NaN
11  01/08/2022       13.58       -0.36          8.73

In [70]:
tabelas_bd()

,name
0,01/01/2022
1,nome_tabela
2,tabela_indices_2
3,selic_url
4,ipca_url
5,igpm_url
6,tabela_selic
7,tabela_ipca
8,tabela_igpm
9,tabela_indices
